In [1]:
import duckdb

In [2]:
con = duckdb.connect("database/sample.db")

In [3]:
# Part A
# Who are the top 10 users with respect to number of songs listened to?
con.sql("""
    select 
        user_name,
        count(*) as num_listens
    from stg.raw
    group by user_name
    order by num_listens desc
    limit 10
""")

┌────────────────┬─────────────┐
│   user_name    │ num_listens │
│    varchar     │    int64    │
├────────────────┼─────────────┤
│ hds            │       46885 │
│ Groschi        │       14959 │
│ Silent Singer  │       13005 │
│ phdnk          │       12859 │
│ 6d6f7274686f6e │       11512 │
│ reverbel       │        8398 │
│ Cl�psHydra     │        8318 │
│ InvincibleAsia │        7804 │
│ cimualte       │        7356 │
│ inhji          │        6344 │
├────────────────┴─────────────┤
│ 10 rows            2 columns │
└──────────────────────────────┘

In [4]:
# How many users did listen to some song on the 1st of March 2019?
con.sql("""
        with base as (
    SELECT 
        user_name,
        STRFTIME(listened_at, '%Y-%m-%d') AS listen_date
    FROM stg.raw
    )
    select count(*) as total_users
        from base
        where listen_date = '2019-03-01'
""")

┌─────────────┐
│ total_users │
│    int64    │
├─────────────┤
│        2491 │
└─────────────┘

In [5]:
# For every user, what was the first song the user listened to?
con.sql("""
    WITH ranked_listens AS (
    SELECT
        user_name,
        track_name,
        ROW_NUMBER() OVER (PARTITION BY user_name ORDER BY listened_at ASC) AS rn
    FROM stg.raw
)
SELECT
    user_name,
    track_name
FROM ranked_listens
WHERE rn = 1
        """)

┌───────────────┬───────────────────────────────────────────────────┐
│   user_name   │                    track_name                     │
│    varchar    │                      varchar                      │
├───────────────┼───────────────────────────────────────────────────┤
│ NichoBI       │ by everyone                                       │
│ arcadian99    │ Crying                                            │
│ augustohp     │ Thrasher                                          │
│ jck2          │ GOLDEN CHORDS                                     │
│ jelera        │ Gin And Juice (Feat. Dat Nigga Daz)               │
│ satanshunger  │ Cartão de Visita                                  │
│ anderson9339  │ Since I've Been Loving You                        │
│ Zastai        │ Fear                                              │
│ lemarac       │ Electronic Performers                             │
│ longas        │ Фильм 2018                                        │
│   ·           │   

In [6]:
# Part B
con.sql("""
    with ranked_listens AS (   
    select
        user_name,
        STRFTIME(listened_at, '%Y-%m-%d') AS date,
        rank() over (partition by user_name order by STRFTIME(listened_at, '%Y-%m-%d')) as listen_rank
    from stg.raw),
        agg_listens AS (
    select 
        user_name,
        date,
        count(*) as number_of_lines,
        row_number() over (partition by user_name order by count(*) desc) as day_rank
    from ranked_listens
    group by user_name, date)
        select
            user_name,
            date,
            number_of_lines
        from agg_listens
        where day_rank < 4
        order by 1,3 desc
""")

┌─────────────────┬────────────┬─────────────────┐
│    user_name    │    date    │ number_of_lines │
│     varchar     │  varchar   │      int64      │
├─────────────────┼────────────┼─────────────────┤
│ 6d6f7274686f6e  │ 2019-01-27 │             204 │
│ 6d6f7274686f6e  │ 2019-01-14 │             198 │
│ 6d6f7274686f6e  │ 2019-03-15 │             196 │
│ Adsky_traktor   │ 2019-01-03 │             109 │
│ Adsky_traktor   │ 2019-01-05 │              99 │
│ Adsky_traktor   │ 2019-01-04 │              86 │
│ AllSparks       │ 2019-01-31 │             113 │
│ AllSparks       │ 2019-01-23 │              81 │
│ AllSparks       │ 2019-01-11 │              72 │
│ AlwinHummels    │ 2019-02-24 │               1 │
│    ·            │     ·      │               · │
│    ·            │     ·      │               · │
│    ·            │     ·      │               · │
│ yellams         │ 2019-02-08 │             160 │
│ zebedeemcdougal │ 2019-02-22 │              82 │
│ zebedeemcdougal │ 2019-02-02 

In [7]:
# Part C
con.sql("""
WITH bounds AS (
    SELECT
        MIN(DATE_TRUNC('day', listened_at)) AS start_date,
        MAX(DATE_TRUNC('day', listened_at)) AS end_date
    FROM stg.raw
),
calendar AS (
    SELECT
        date
    FROM (
        SELECT start_date, end_date FROM bounds
    ) AS b,
    UNNEST(GENERATE_SERIES(b.start_date, b.end_date, INTERVAL 1 DAY)) AS t(date)
),
all_users AS (
    SELECT COUNT(DISTINCT user_name) AS total_users
    FROM stg.raw
),
active AS (
    SELECT
        STRFTIME(c.date, '%Y-%m-%d') AS date,
        COUNT(DISTINCT r.user_name) AS number_active_users
    FROM calendar c
    LEFT JOIN stg.raw r
        ON r.listened_at >= c.date - INTERVAL 6 DAY
       AND r.listened_at <  c.date + INTERVAL 1 DAY
    GROUP BY c.date
)
SELECT
    date,
    number_active_users,
    ROUND(100.0 * number_active_users / total_users, 2) AS percentage_active_users
FROM active, all_users
ORDER BY date;
""")

┌────────────┬─────────────────────┬─────────────────────────┐
│    date    │ number_active_users │ percentage_active_users │
│  varchar   │        int64        │         double          │
├────────────┼─────────────────────┼─────────────────────────┤
│ 2019-01-01 │                  72 │                   35.82 │
│ 2019-01-02 │                  95 │                   47.26 │
│ 2019-01-03 │                 103 │                   51.24 │
│ 2019-01-04 │                 107 │                   53.23 │
│ 2019-01-05 │                 108 │                   53.73 │
│ 2019-01-06 │                 110 │                   54.73 │
│ 2019-01-07 │                 114 │                   56.72 │
│ 2019-01-08 │                 113 │                   56.22 │
│ 2019-01-09 │                 115 │                   57.21 │
│ 2019-01-10 │                 115 │                   57.21 │
│     ·      │                   · │                     ·   │
│     ·      │                   · │                   

In [8]:
con.close()